# Poisson Equation in Parallel

NGSolve can be executed on a cluster using the MPI message passing interface.
You can download [poisson_mpi.py](poisson_mpi.py) and run it as

> mpirun -np 4 python3 poisson_mpi.py 

The solution is saved such that we can visualize it with [drawsolution.py](drawsolution.py)

> netgen drawsolution.py

For proper parallel execution, Netgen/NGSolve must be configured with '-DUSE_MPI=ON'. Recent binaries for Linux and Mac are built with MPI support (?). If you are unsure if your Netgen/NGSolve supports MPI, look for output like "Including MPI version 3.1" douring Netgen startup.

## MPI-parallel execution using ipyparallel

In the jupyter-tutorials we use ipyparallel. Please consult https://ipyparallel.readthedocs.io for installation.

On my notebook I have created the profile once by the command

>ipython profile create --parallel --profile=mpi

Since I have only two physical cores, I edited the file
.ipython/profile_mpi/ipcluster_config.py to allow for oversubscribtion:

> c.MPILauncher.mpi_args = ['--oversubscribe']



I start the cluster via

> ipcluster start --engines=MPI -n 4 --profile=mpi




In jupyter, we can then connect to the cluster via

In [1]:
from ipyparallel import Client
c = Client(profile='mpi')
c.ids

[0, 1, 2, 3]

We use mpi4py https://mpi4py.readthedocs.io/ for issuing MPI calls from Python. The %%px syntax magic causes parallel execution of that cell:

In [2]:
%%px 
from mpi4py import MPI
comm = MPI.COMM_WORLD
print (comm.rank, comm.size)

[stdout:0] 0 4
[stdout:1] 1 4
[stdout:2] 2 4
[stdout:3] 3 4


The master process (rank==0) generates the mesh, and distributes it within the group of processes defined by the communicator. All other ranks receive a part of the mesh. The function mesh.GetNE(VOL) returns the local number of elements:

In [3]:
%%px
from ngsolve import *
from netgen.geom2d import unit_square

if comm.rank == 0:
    mesh = Mesh(unit_square.GenerateMesh(maxh=0.1).Distribute(comm))
else:
    mesh = Mesh(netgen.meshing.Mesh.Receive(comm))
print (mesh.GetNE(VOL))

[stdout:0] 0
[stdout:1] 75
[stdout:2] 76
[stdout:3] 79


We can define spaces, bilinear and linear forms, gridfunctions the same way as in sequential mode. But now, the degrees of freedom are distributed on the cluster following the distribution of the mesh. The finite element spaces defines how the dofs match together.

In [4]:
%%px
fes = H1(mesh, order=3, dirichlet=".*")
u,v = fes.TnT()

a = BilinearForm(grad(u)*grad(v)*dx)
pre = Preconditioner(a, "local")
a.Assemble()

f = LinearForm(1*v*dx).Assemble()
gfu = GridFunction(fes)

from ngsolve.krylovspace import CGSolver
inv = CGSolver(a.mat, pre.mat, printing=comm.rank==0, maxsteps=200, tol=1e-8)
gfu.vec.data = inv*f.vec

[stdout:0] 
iteration 0 error = 0.05333923409291168
iteration 1 error = 0.07404604143104701
iteration 2 error = 0.06370425458795258
iteration 3 error = 0.04919124251788337
iteration 4 error = 0.0482965254214221
iteration 5 error = 0.0311069125544914
iteration 6 error = 0.024935643398100362
iteration 7 error = 0.01784194991245131
iteration 8 error = 0.013817492065755404
iteration 9 error = 0.01142376931907761
iteration 10 error = 0.008393867618520493
iteration 11 error = 0.004233406891736331
iteration 12 error = 0.0019377308491299833
iteration 13 error = 0.0014085555318202965
iteration 14 error = 0.0009913437971717843
iteration 15 error = 0.000641221732154126
iteration 16 error = 0.00044876370080339906
iteration 17 error = 0.00029026903025157816
iteration 18 error = 0.00017121515214383356
iteration 19 error = 0.00013131363792643478
iteration 20 error = 0.0001018063078127641
iteration 21 error = 6.493748016473814e-05
iteration 22 error = 3.7473288652453216e-05
iteration 23 error = 2.4894

Parallel pickling allows to serialize the distributed solution and transfer it to the client:

In [5]:
%%px
netgen.meshing.SetParallelPickling(True)

In [7]:
gfu = c[:]["gfu"]
from ngsolve.webgui import Draw
Draw (gfu[0])

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2007-133-g65ffe4820', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2…

We can also visualize the sub-domains obtained by the automatic partitioning:

In [8]:
%%px
fesL2 = L2(mesh, order=0)
gfL2 = GridFunction(fesL2)
gfL2.vec.local_vec[:] = comm.rank

In [9]:
gfL2 = c[:]["gfL2"]
Draw (gfL2[0])

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2007-133-g65ffe4820', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2…